<a href="https://colab.research.google.com/github/princessivy/course/blob/main/model_dl_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Packages

In [2]:
!pip install ndjson --quiet
!pip install langdetect --quiet

     |████████████████████████████████| 981 kB 4.2 MB/s 


In [3]:
import pandas as pd
import ndjson
import gzip
from google.colab import drive
from pathlib import Path
from xgboost import XGBRegressor
from langdetect import detect

from sklearn.model_selection import cross_val_score


In [8]:
## for sentiment
from textblob import TextBlob

# Load Datasets

In [3]:
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
path_train = Path('/content/drive/MyDrive/Preprocessed_data/de eng_train_preprocessed/df_train_german.ndjson')

with open(path_train) as f:
    data = ndjson.load(f)
df_train = pd.DataFrame(data)

# Lang-Tag vervollständigen & eng/ger speichern

In [ ]:
#df_train[df_train.lang_code == 'NaN']

for x in range(len(df_train)):
  if df_train['lang_code'][x] == 'NaN':
    df_train['lang_code'][x] = detect(' '.join(df_train['title'][x]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df_train_german = df_train[df_train.lang_code == 'de'].copy()

In [ ]:
#index 

df_train_german = df_train_german.reset_index(drop=True)

In [ ]:
df_train_english

,url,industry,industry_label,img_alt,h1,h2,h3,strong,bold,lang_code,figcaption,pure_text,title,concatenated
0,http://www.beumergroup.com,135,Mechanical or Industrial Engineering,Ship Loader Vimeo Vimeo Vimeo Ship Loader,Welcome to Beumer Group,Intralogistic Solutions For Your Industry Cust...,Airport Baggage Handling Building Material Cem...,,,en,Alexandrapurchasing Department Maiksite Superv...,Beumer Group International Quality Leader Intr...,Beumer Group International Quality Leader Intr...,Ship Loader Vimeo Vimeo Vimeo Ship Loader Welc...
1,http://www.aurigavision.com,13,Medical Practice,Service Kandidat Job Healthcare Recruitment Ex...,Aurigavision Partner Recruitment Executive Sea...,New Service Kandidat Job Healthcare Recruitmen...,Job Registration Requirement Optimizing Videoc...,Employers are Required,,en,,Home Aurigavision De Home Job Healthcareaktuel...,Home Aurigavision,Service Kandidat Job Healthcare Recruitment Ex...
2,http://www.cis-inspector.com,135,Mechanical or Industrial Engineering,Cis Asme Tüv Thüringen Group Asme Code Agency ...,,,,,,en,,﻿ Cis Asme Code Inspection Courses Design Calc...,Cis Asme Code Inspection Courses Design Calcul...,Cis Asme Tüv Thüringen Group Asme Code Agency ...
3,http://mynaric.com,8,Telecommunications,Close Icon Close Icon Hover State,Connectivity For The skies Beyond,our Mission what we Do a History Of Growth Pre...,Satellite Constellations Inter Satellite Link ...,Optical Inter Satellite links,,en,,Mynaric Connectivity For The skies And Beyond ...,Mynaric Connectivity For The skies And Beyond,Close Icon Close Icon Hover State Connectivity...
4,http://www.MAG45.com,116,Logistics and Supply Chain,mögen Industrial Supply Solutions En english D...,The Reference Integrated Supply mögen Solutions,White pap Three perspectives On Effective Bom ...,Succesful Webinar On Supplier Declaration Thre...,The Reference Integrated Supply,,en,we are mögen,mögen Home <﻿ Iframe Src Https Www Googletagma...,mögen Home,mögen Industrial Supply Solutions En english D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6581,http://www.pscgroup-lb.com,48,Construction,,,Welcome to Psc Group s a L after Sales what we...,Contact Us,For Every Job,,en,,Psc Group Psc Group For Every Job we have The ...,Psc Group Psc Group,Welcome to Psc Group s a L after Sales what ...
6582,http://www.fides.ch,43,Financial Services,Multi Bank Solutions Multi Bank Lösung Multi B...,Reach Further Connect Fast,we Centralize And Simplify Multi Bank Connecti...,how we help Let s Connect,,,en,,Fides Treasury Services Ltd Reach Further Conn...,Fides Treasury Services Ltd Reach Further Conn...,Multi Bank Solutions Multi Bank Lösung Multi B...
6583,http://www.buildarocket.de,80,Marketing and Advertising,Esports Marketing Dedicated to Partner Esports...,Your Launching Padinto The Esports Universe,Your Missionis our Mission New Projectswe ve B...,Announcing partnership With Sportfive Happy Ne...,,,en,Dedicated to our Partner Unparalleled experien...,Build a rocken Esports Agency Consulting Marke...,Build a rocken Esports Agency Consulting Marke...,Esports Marketing Dedicated to Partner Esports...
6584,http://www.starkhomes.com,44,Real Estate,Analytics Stark Stark Stark Ad Image The Marke...,,what s Your Home Worth Move to The Life you Wa...,,,,en,,South Central Wisconsin Real Estate Stark Comp...,South Central Wisconsin Real Estate Stark Comp...,Analytics Stark Stark Stark Ad Image The Marke...


In [106]:
datasaver_to_ndjson(df_train_x, 'df_train_de_sentiment')

In [90]:
def datasaver_to_ndjson(df, name):
  # create flat list in dict form: {'col1': 'value', 'col2': 'value', ...} from df
  flat_list_back = []
  for i in range(len(df)):
    line = df.loc[i].to_dict()
    #line['industry'] = str(line['industry']) # use if idustry number (e.g. 13) should be enclosed in '' (e.g. '13')
    flat_list_back.append([line])

  filename = str(name) + '.ndjson'

  # https://stackoverflow.com/questions/21058935/python-json-loads-shows-valueerror-extra-data
  with open(filename, mode='w') as f:
    for item in flat_list_back:
      f.write('{}\n'.format(ndjson.dumps(item))) 

# Sentiment Analysis

In [14]:
#print(df_train['pure_text'][0].apply(lambda x: TextBlob(x).sentiment.polarity))



print(TextBlob(df_train['pure_text'][5]).sentiment.polarity)

0.0953168044077135


In [92]:
df_train_x = df_train.copy()

In [18]:
for x in range(1, 3):
  print(TextBlob(df_train['pure_text'][x]).sentiment.polarity)

-0.10795454545454547
0.01


In [119]:
df_train_x['sentiment_analysis_test'] =''

In [120]:
for x in range(0, len(df_train_x)):
  df_train_x.sentiment_analysis_test[x] = round(TextBlob(df_train_x['h1'][x]).sentiment.polarity,2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
df_train_x['pure_text'] = df_train_x['pure_text'].astype(str)

In [136]:
df_train_x.groupby('industry_label')['sentiment_analysis_test'].mean()

industry_label
Automotive                              0.571786
Construction                            0.559210
Consumer Goods                          0.571234
Financial Services                      0.565933
Human Resources                         0.567829
Information Technology and Services     0.565472
Insurance                               0.567530
Legal Services                          0.553263
Leisure, Travel & Tourism               0.609034
Logistics and Supply Chain              0.564007
Management Consulting                   0.560519
Marketing and Advertising               0.567129
Mechanical or Industrial Engineering    0.556005
Medical Practice                        0.570085
Real Estate                             0.569332
Recreational Facilities and Services    0.581462
Renewables & Environment                0.571447
Telecommunications                      0.564032
Wholesale                               0.572635
Name: sentiment_analysis_test, dtype: float64

In [123]:
from sklearn import preprocessing

In [124]:
x = df_train_x.sentiment_analysis

In [134]:
for x in range(0,len(df_train_x)):
  df_train_x.sentiment_analysis_test[x] = (df_train_x.sentiment_analysis[x]-df_train_x.sentiment_analysis.min())/(df_train_x.sentiment_analysis.max()-df_train_x.sentiment_analysis.min())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [135]:
df_train_x

,url,industry,industry_label,img_alt,h1,h2,h3,strong,bold,lang_code,figcaption,pure_text,title,concatenated,sentiment_analysis,sentiment_analysis_test
0,http://www.kaiser-personal-service.de,137,Human Resources,,,aktuell Stellenangebot,,,,de,,Kaiser Personal Service Impressum Datumenschut...,Kaiser Personal Service,aktuell Stellenangebot de Kaiser Persona...,0.0,0.5
1,http://www.brandseven.com,96,Information Technology and Services,Brandseven Energy Cockpit Innovation Lab Brand...,Maßgeschneiderte Vertriebstool Stadtwerk Evu s,Brandseven Go Remote Biohymethcontrol richtig ...,Energy Cockpit Edition,Beispiel Gedankenexperiment Intervention Ergebnis,,de,,Brandseven digital Lösung Energiewirtschaft se...,Brandseven digital Lösung Energiewirtschaft se...,Brandseven Energy Cockpit Innovation Lab Brand...,-0.11,0.445
2,http://www.novum-rgi.de,96,Information Technology and Services,Slider Übernahme Flexperto Jpg,seit Jahr entwickeln Softwarelösung Leidenscha...,Unternehmen Novum Novum Rgi Standort Karriere ...,Novum Rgi Zukunftssichere it Lösung ​ entwicke...,,,de,,Novum Rgi Zukunftssichere it Lösung Versicheru...,Novum Rgi Zukunftssichere it Lösung Versicheru...,Slider Übernahme Flexperto Jpg seit Jahr entwi...,0.01,0.505
3,http://www.timmermannpartners.com,11,Management Consulting,Timmermann Partner Symbol Timmermann Partner Z...,,Chapp holen Fach Gold German Stevie Awards auß...,Profil Michael Timmermann professionell Hinter...,Cmo Geschäftsfeldleiter Leiterin Personalentwi...,,de,,Timmermann Partner Beratung Startseite Experti...,Timmermann Partner Beratung,Timmermann Partner Symbol Timmermann Partner Z...,0.19,0.595
4,http://www.spk-row-ohz.de,43,Financial Services,zurück Startseite zurück Startseite zurück Sta...,weil ’ s Geld gehen weil ’ s Geld gehen,,,,,de,,Finanzpartner Sparkasse Rotenburg Osterholz Pr...,Finanzpartner Sparkasse Rotenburg Osterholz,zurück Startseite zurück Startseite zurück Sta...,0.25,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16804,http://www.connectcompetence.net,11,Management Consulting,Https Www Co Connectcompetence Https Www Co Co...,aktuell Stelle Connect Competence Experte Pers...,Kurzvideo Gehaltsstudie Tirol Ansehen,Initiativbewerbung Stellenangebot Gehaltskoste...,,,de,,willkommen Connect Competence Personalberatung...,willkommen Connect Competence Personalberatung...,Https Www Co Connectcompetence Https Www Co Co...,0.01,0.505
16805,http://www.nfon.com,8,Telecommunications,Nfon Ag Nfon Kunde Nfon Kunde Nfon Kunde Nfon ...,Nfon neu Freiheit Businesskommunikation,zufrieden Kunde Vertrauen Nfon entdecken Cloud...,,,,de,,Nfon neu Freiheit Businesskommunikation Telefo...,Nfon neu Freiheit Businesskommunikation,Nfon Ag Nfon Kunde Nfon Kunde Nfon Kunde Nfon ...,0.0,0.5
16806,http://www.hotel-konsul.de,30,"Leisure, Travel & Tourism",Hotel Konsul Logo Avatar Avatar Avatar Avatar ...,Konsul Hotel,Zimmer Seminar Tagungsraum aktuell Empfangsmit...,Konsul Hotel willkommen Konsul willkommen Haus...,Halle Leipzig Dessau Wörlitz Airport Leipzig H...,,de,,Hotel Konsul Hotel Konsul Inhalt springen Semi...,Hotel Konsul Hotel Konsul,Hotel Konsul Logo Avatar Avatar Avatar Avatar ...,0.38,0.69
16807,http://www.suva.ch,42,Insurance,Leben schön solange passieren Dsc Jpg Safety A...,,Videos New,,,,de,,Suva Unfallversicherung Prävention Versicherun...,Suva Unfallversicherung Prävention Versicherun...,Leben schön solange passieren Dsc Jpg Safety A...,0.15,0.575


In [130]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,16769,16770,16771,16772,16773,16774,16775,16776,16777,16778,16779,16780,16781,16782,16783,16784,16785,16786,16787,16788,16789,16790,16791,16792,16793,16794,16795,16796,16797,16798,16799,16800,16801,16802,16803,16804,16805,16806,16807,16808
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Build Model

In [ ]:
hyperparameter tuning: gridsearch(!), randomised search
# https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn
# https://www.datacamp.com/community/tutorials/xgboost-in-python#apply

fastai
# https://fastai1.fast.ai/tabular.html

mega?
# https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794



# Test für mein Verständnis

In [142]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [143]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [144]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [145]:
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(y, y_pred)

print(np.sqrt(mse))

[16:54:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
36.271203581682585


# DL - word embedding

In [ ]:
df_train_german.concatenated[0]

'  aktuell Stellenangebot    de  Kaiser Personal Service Impressum Datumenschutz Bewerbung Kp Darmstadt Kp Rheinland suchen Arbeit Staplerfahrer bewerben suchen Arbeit Verkäufer bewerben suchen Arbeit Telefonist bewerben aktuell Stellenangebot Standort wählen Kp Darmstadt Kp Rheinland Kcm Königsfeld Kommunikationselektronik Fernmeldemonteur Elektriker Elektronik Elektronik Raum Darmstadtab Sofortkennziffer Dafmjob Detailsstaplerfahrer Staplerfahrer Biebesheimab Sofortkennziffer Dastjob Detailsmitarbeiter Oberflächentechnik Helfer groß Umstadtab Sofortkennziffer Guotjob Detailssachbearbeiter Tz Zollkenntnissensachbearbeiter Troisdorfab Sofortkennziffer Sbzolljob Detailskommissionierer Kommissionierer groß Gerauab Sofortkennziffer Daggjob Detailsmaschine Anlagenführer Maschinenbediener Bonnab Sofortkennziffer Maanfüjob Detailsspeditionskaufmann Speditionskaufmann Großraum Bonnab Sofortkennziffer Spekfmjob Detailskommissionierer Kommissionierer Weiterstadtab Sofortkennziffer Dakojob Detai

In [ ]:
df_train_german.head(1)

,url,industry,industry_label,img_alt,h1,h2,h3,strong,bold,lang_code,figcaption,pure_text,title,concatenated
0,http://www.kaiser-personal-service.de,137,Human Resources,,,aktuell Stellenangebot,,,,de,,Kaiser Personal Service Impressum Datumenschut...,Kaiser Personal Service,aktuell Stellenangebot de Kaiser Persona...


In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 3.5 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 6.8 MB 15.9 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 


In [ ]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [ ]:
df_train_german

,url,industry,industry_label,img_alt,h1,h2,h3,strong,bold,lang_code,figcaption,pure_text,title,concatenated
0,http://www.kaiser-personal-service.de,137,Human Resources,,,aktuell Stellenangebot,,,,de,,Kaiser Personal Service Impressum Datumenschut...,Kaiser Personal Service,aktuell Stellenangebot de Kaiser Persona...
1,http://www.brandseven.com,96,Information Technology and Services,Brandseven Energy Cockpit Innovation Lab Brand...,Maßgeschneiderte Vertriebstool Stadtwerk Evu s,Brandseven Go Remote Biohymethcontrol richtig ...,Energy Cockpit Edition,Beispiel Gedankenexperiment Intervention Ergebnis,,de,,Brandseven digital Lösung Energiewirtschaft se...,Brandseven digital Lösung Energiewirtschaft se...,Brandseven Energy Cockpit Innovation Lab Brand...
2,http://www.novum-rgi.de,96,Information Technology and Services,Slider Übernahme Flexperto Jpg,seit Jahr entwickeln Softwarelösung Leidenscha...,Unternehmen Novum Novum Rgi Standort Karriere ...,Novum Rgi Zukunftssichere it Lösung ​ entwicke...,,,de,,Novum Rgi Zukunftssichere it Lösung Versicheru...,Novum Rgi Zukunftssichere it Lösung Versicheru...,Slider Übernahme Flexperto Jpg seit Jahr entwi...
3,http://www.timmermannpartners.com,11,Management Consulting,Timmermann Partner Symbol Timmermann Partner Z...,,Chapp holen Fach Gold German Stevie Awards auß...,Profil Michael Timmermann professionell Hinter...,Cmo Geschäftsfeldleiter Leiterin Personalentwi...,,de,,Timmermann Partner Beratung Startseite Experti...,Timmermann Partner Beratung,Timmermann Partner Symbol Timmermann Partner Z...
4,http://www.spk-row-ohz.de,43,Financial Services,zurück Startseite zurück Startseite zurück Sta...,weil ’ s Geld gehen weil ’ s Geld gehen,,,,,de,,Finanzpartner Sparkasse Rotenburg Osterholz Pr...,Finanzpartner Sparkasse Rotenburg Osterholz,zurück Startseite zurück Startseite zurück Sta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16804,http://www.connectcompetence.net,11,Management Consulting,Https Www Co Connectcompetence Https Www Co Co...,aktuell Stelle Connect Competence Experte Pers...,Kurzvideo Gehaltsstudie Tirol Ansehen,Initiativbewerbung Stellenangebot Gehaltskoste...,,,de,,willkommen Connect Competence Personalberatung...,willkommen Connect Competence Personalberatung...,Https Www Co Connectcompetence Https Www Co Co...
16805,http://www.nfon.com,8,Telecommunications,Nfon Ag Nfon Kunde Nfon Kunde Nfon Kunde Nfon ...,Nfon neu Freiheit Businesskommunikation,zufrieden Kunde Vertrauen Nfon entdecken Cloud...,,,,de,,Nfon neu Freiheit Businesskommunikation Telefo...,Nfon neu Freiheit Businesskommunikation,Nfon Ag Nfon Kunde Nfon Kunde Nfon Kunde Nfon ...
16806,http://www.hotel-konsul.de,30,"Leisure, Travel & Tourism",Hotel Konsul Logo Avatar Avatar Avatar Avatar ...,Konsul Hotel,Zimmer Seminar Tagungsraum aktuell Empfangsmit...,Konsul Hotel willkommen Konsul willkommen Haus...,Halle Leipzig Dessau Wörlitz Airport Leipzig H...,,de,,Hotel Konsul Hotel Konsul Inhalt springen Semi...,Hotel Konsul Hotel Konsul,Hotel Konsul Logo Avatar Avatar Avatar Avatar ...
16807,http://www.suva.ch,42,Insurance,Leben schön solange passieren Dsc Jpg Safety A...,,Videos New,,,,de,,Suva Unfallversicherung Prävention Versicherun...,Suva Unfallversicherung Prävention Versicherun...,Leben schön solange passieren Dsc Jpg Safety A...


# XGBoost

In [ ]:
import numpy as np

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

from sklearn.feature_extraction.text import TfidfVectorizer

# einlesen
from sklearn.compose import ColumnTransformer

In [150]:
X = df_train_x[['concatenated', 'sentiment_analysis']]
y = df_train_x['industry']

In [157]:
# wende tfidf an
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_x['concatenated'])

In [178]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)

In [160]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(train_X, train_y)

y_pred = xgb_model.predict(val_X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [183]:
# confusion matrix
print(confusion_matrix(y_pred, val_y))

[[ 19   0   1   0   0   0   0   0   0   0   0   0   1   0   0   1   0   0
    0]
 [  0  82   4   0   0   0   0   0   1   0   0   0   1   0   0   0   0   2
    0]
 [  5   3 241   5   2   2   1   1   7   3   3   2  10  36   2   4   3  18
    3]
 [  0   1   3 103   2   0   2   1   3   0   0   1   3  11   0   3   1   1
    0]
 [  2   0   2   1  56   0   0   0   0   1   4   4   0   1   0   6  17   0
    2]
 [  0   0   0   0   0  40   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   3  56   0   0   0   0   1   3   0   0   7   0   0
    0]
 [  0   0   1   1   0   0   0  35   2   0   0   1   1   1   1   0   0   0
    0]
 [  0   1   3   0   0   0   0   2 172   2   0   0   4  10   1   0   0   0
    1]
 [  1   2   3   0   0   1   0   0   7 114   8   0   1   0   0   0   0   1
    0]
 [  2   1   3   0   3   0   0   0   1   6 128   0   0   1   0   7   7   1
    8]
 [  0   0   0   0   2   0   0   2   2   1   0  90   4   3   1   5  12   1
    0]
 [  1   0  26   4   0   3   

In [182]:
score = xgb_model.score(val_X, val_y)
score

0.7328098976921247

In [167]:
from sklearn import metrics
import numpy as np

In [179]:
logit_report = metrics.classification_report(val_y, y_pred, labels=val_y, output_dict=True)

In [181]:
pd.DataFrame(logit_report).T

,precision,recall,f1-score,support
43,0.877551,0.767857,0.819048,2.240000e+02
96,0.729389,0.860412,0.789501,8.740000e+02
11,0.686610,0.593596,0.636724,4.060000e+02
25,0.583333,0.368421,0.451613,1.520000e+02
116,0.924242,0.762500,0.835616,8.000000e+01
8,0.863636,0.351852,0.500000,5.400000e+01
135,0.648459,0.823843,0.725705,5.620000e+02
80,0.762980,0.761261,0.762120,4.440000e+02
48,0.761905,0.691892,0.725212,1.850000e+02
137,0.791209,0.610169,0.688995,1.180000e+02
